In [ ]:
# Demo notebook- agentic rag

# A teljes RAG pipeline LangChainnel

# 1 Loader 2 Chunker 3 Embeddings (HuggungFace) 4 Chroma 5 Retreiver 6 RetrievalQA + LLM


In [40]:
import sys
from pathlib import Path

root = Path.cwd().parent
if str(root) not in sys.path:
    sys.path.append(str(root))

In [41]:
#Környezet tesztelése + importok 

from src.agentic_rag.loader import load_sample
from src.agentic_rag.chunker import simple_chunk
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline


In [42]:
#Loader

text = load_sample()

In [43]:
#Chunker

chunks = simple_chunk(text,max_words=10)
for i, chun in enumerate(chunks[:3], 1):
    print(f"{i}.", chun)

1. Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can
2. fly high in the sky. Fish swim in the water.


In [44]:

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")
vectorstore = Chroma(collection_name="rag_collection_demo",embedding_function=embedding_model)
vectorstore.add_texts(chunks)
retiever = vectorstore.as_retriever(search_kwargs={"k":2})
print("Done.")

Done.


In [47]:
# Retriever teszt

question = "what is your opinion on cats?"
doc = retiever.get_relevant_documents(question)

print(question)
for i, d in enumerate(doc, 1):
    print(f"{i}.", d.page_content)


what is your opinion on cats?
1. Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can
2. Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can


In [ ]:
hf_generator =  pipeline("text2text-generation", model="google/flan-t5-base")
llm = HuggingFacePipeline(pipeline=hf_generator)

quan = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retiever,
    chain_type="stuff"
)

answer = quan.run(question)
print(answer)

Device set to use cpu


arrogant


In [49]:
questions = [
    "Which animal can swim?",
    "what do you think about dogs?",
    "what is your opinion on cats?"
]

for i in questions:
    doc = retiever.get_relevant_documents(i)
    answers = quan.run(i)
    print(f"\n {i}")
    print(f"Retrieved: {[d.page_content for d in doc]}")
    print(f"Answer: {answers}")


 Which animal can swim?
Retrieved: ['fly high in the sky. Fish swim in the water.', 'fly high in the sky. Fish swim in the water.']
Answer: Fish

 what do you think about dogs?
Retrieved: ['Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can', 'Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can']
Answer: i think dogs are much friendlier than cats

 what is your opinion on cats?
Retrieved: ['Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can', 'Cats sometimes behave arrogantly. Dogs are much friendlier. Birds can']
Answer: arrogant
